In [1]:
import datajoint as dj
import numpy as np
import time

import pandas as pd
import seaborn as sns
sns.set() #sets the asethetics parameters in what set

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
#creates the table with spine data without the spine label
#how to filter out the spine proportion

#have to first remove the ones that have only spine synapses


neurite_targets_no_spine = (pinky.NeuriteTargetsRevised() & "spine_num<total").proj(spine_head_prop="spine_head_prop/(1-spine_prop)",
                                                         spine_neck_prop="spine_neck_prop/(1-spine_prop)",
                                                         shaft_prop="shaft_prop/(1-spine_prop)",
                                                         non_spine_prop="non_spine_prop/(1-spine_prop)",
                                                         total="total-spine_num")



In [4]:
#different lengths based on the number of synapses threshold

print(len(neurite_targets_no_spine))
print(len(neurite_targets_no_spine & "total>12"))
print(len(neurite_targets_no_spine & "total>10")) 
print(len(neurite_targets_no_spine & "total>8")) 
print(len(neurite_targets_no_spine & "total>6")) 

30088
1666
2912
5312
10102


In [5]:
neurite_targets_no_spine_6 = (neurite_targets_no_spine & "total>6")
neurite_targets_no_spine_6

segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_head_prop calculated attribute,spine_neck_prop calculated attribute,shaft_prop calculated attribute,non_spine_prop calculated attribute,total calculated attribute
3,648518346341351514,0.6666666942614097,0.11111111295076065,0.2222222259015213,0.0,9.0
3,648518346341351518,0.8571428656578064,0.1428571492433548,0.0,0.0,7.0
3,648518346341351525,0.75,0.0833333358168602,0.1666666716337204,0.0,12.0
3,648518346341351631,0.8181818127632141,0.09090909361839294,0.09090909361839294,0.0,22.0
3,648518346341351695,0.6666666865348816,0.2222222238779068,0.1111111119389534,0.0,9.0
3,648518346341351719,0.8571428571428571,0.0,0.14285714285714285,0.0,7.0
3,648518346341351767,0.7777777658200559,0.11111111295076065,0.11111111295076065,0.0,9.0
3,648518346341351896,0.8181818127632141,0.09090909361839294,0.09090909361839294,0.0,11.0
3,648518346341351904,0.0,0.1428571492433548,0.8571428656578064,0.0,7.0
3,648518346341351978,0.8749999883584678,0.0,0.12500000232830644,0.0,8.0


In [6]:
#manually find the groups
axon_group = neurite_targets_no_spine_6 & "axon_prop>0"
print("Axon group = " + str(len(axon_group)))
soma_group = (neurite_targets_no_spine_6 - axon_group.proj()) & "soma_prop>=0.25"
print("Soma group = " + str(len(soma_group)))

InternalError: (1054, "Unknown column 'axon_prop' in 'where clause'")

In [ ]:
#remove the axon and soma groups
neurite_targets_no_spine_6_without_axon_soma = neurite_targets_no_spine_6 - axon_group.proj() - soma_group.proj()
len(neurite_targets_no_spine_6_without_axon_soma)/len(neurite_targets_no_spine_6)

In [ ]:
#get the portions of groups first
head_group = neurite_targets_no_spine_6 & "spine_head_prop>0.6"
head_group

In [ ]:
#get the portions of groups first
shaft_group = neurite_targets_no_spine_6 & "spine_head_prop<=0.6" & "shaft_prop>=0.5"
shaft_group

In [ ]:
unused = (neurite_targets_no_spine_6 - shaft_group.proj()) - head_group.proj()
unused

In [ ]:
#check that the lengths are the same
print(len(neurite_targets_no_spine_6))
print(len(shaft_group) + len(head_group) + len(unused))

In [ ]:

#pull down the head data and sort
head_data = head_group.fetch(as_dict=True)

head_df = pd.DataFrame(head_data)
head_df_base = head_df.drop(columns=['segmentation', 'segment_id',"total","non_spine_prop","spine_neck_prop"])

spine_df_base_head_shaft_sort = head_df_base.sort_values(['spine_head_prop','shaft_prop'], ascending=[False,True])
spine_df_base_head_shaft_sort



In [ ]:
#heat map of spine heads
spine_head_shaft_sort = sns.heatmap(spine_df_base_head_shaft_sort)

In [ ]:

#pull down the head data and sort
shaft_data = shaft_group.fetch(as_dict=True)

shaft_df = pd.DataFrame(shaft_data)
shaft_df_base = shaft_df.drop(columns=['segmentation', 'segment_id',"total","non_spine_prop","spine_neck_prop"])

shaft_df_base_head_shaft_sort = shaft_df_base.sort_values(['shaft_prop','spine_head_prop'], ascending=[True,False])
#shaft_df_base_head_shaft_sort



In [ ]:
sns.heatmap(shaft_df_base_head_shaft_sort)

In [ ]:

#pull down the head data and sort
unused_data = unused.fetch(as_dict=True)

unused_df = pd.DataFrame(unused_data)
unused_df_base = unused_df.drop(columns=['segmentation', 'segment_id',"total","non_spine_prop","spine_neck_prop"])

unused_df_base_head_shaft_sort = unused_df_base.sort_values(['shaft_prop','spine_head_prop'], ascending=[True,False])
unused_df_base_head_shaft_sort



In [ ]:
sns.heatmap(unused_df_base)

In [ ]:
#### CAN'T END UP DOING NANS!
# nans = pd.DataFrame([dict(spine_head_prop=np.nan,shaft_prop=np.nan)]*100)
# sns.heatmap(nans)

In [ ]:
# total = spine_df_base_head_shaft_sort +  shaft_df_base_head_shaft_sort + unused_df_base_head_shaft_sort



# total = pd.concat(spine_df_base_head_shaft_sort,
#           unused_df_base_head_shaft_sort,
#          shaft_df_base_head_shaft_sort)

In [ ]:
sns.heatmap(unused_df_base_head_shaft_sort)

In [ ]:
sns.heatmap(pd.concat([spine_df_base_head_shaft_sort,
          unused_df_base_head_shaft_sort,
         shaft_df_base_head_shaft_sort]))

# Creating the Spine Groups

In [ ]:
#look up datajoint table that has the clusters
@schema
class SpineClustersKey(dj.Lookup):
    definition="""
    cluster_id : tinyint unsigned #numeric label of cluster
    ---
    description : varchar(16) #descriptive name of cluster
    threshold   : float #the proportion cut off for the group based on the category
    """
    

In [ ]:
cluster_descriptions = ["Soma","Axon","Spine Head","Shaft","Unused"]
cluster_thresholds = [0.25,0.0001,0.6,0.5,0]

clusters_dict = [dict(cluster_id=i,description=cd,threshold=ct) for i,(cd,ct) in enumerate(zip(cluster_descriptions,cluster_thresholds))]
clusters_dict

In [ ]:
SpineClustersKey.insert(clusters_dict,skip_duplicates=True)

In [ ]:
SpineClustersKey()

In [ ]:
## Creates the table that holds all of the segment groups
@schema
class SpineClusters(dj.Manual):
    definition="""
    -> pinky.Neurite
    ---
    ->SpineClustersKey
    """

In [ ]:
possible_groups = [soma_group,axon_group,head_group,shaft_group,unused]
descriptors = [0,1,2,3,4]
for i,table in enumerate(possible_groups):
    new_table = table.proj(cluster_id=str(descriptors[i]))
    SpineClusters.insert(new_table,skip_duplicates=True)
    